<a href="https://colab.research.google.com/github/Reuben17/Data_cleaning/blob/main/task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obtaining the Dataset

In [49]:
import pandas as pd
import io
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split

In [50]:
from google.colab import files
uploaded = files.upload()

Saving modified_data.csv to modified_data (1).csv


In [51]:
dataset = pd.read_csv(io.BytesIO(uploaded['modified_data.csv']))
print(dataset)

        Id  MSSubClass MSZoning  LotFrontage  LotArea Street LotShape  \
0        1          60       RL         65.0     8450   Pave      Reg   
1        2          20       RL         80.0     9600   Pave      Reg   
2        3          60       RL         68.0    11250   Pave      IR1   
3        4          70       RL         60.0     9550   Pave      IR1   
4        5          60       RL         84.0    14260   Pave      IR1   
...    ...         ...      ...          ...      ...    ...      ...   
1455  1456          60       RL         62.0     7917   Pave      Reg   
1456  1457          20       RL         85.0    13175   Pave      Reg   
1457  1458          70       RL         66.0     9042   Pave      Reg   
1458  1459          20       RL         68.0     9717   Pave      Reg   
1459  1460          20       RL         75.0     9937   Pave      Reg   

     LandContour Utilities LotConfig  ... EnclosedPorch 3SsnPorch ScreenPorch  \
0            Lvl    AllPub    Inside  ... 

# Dataset Details and Dropping

In [52]:
dataset.head(2)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,...,0,0,0,0,0,5,2007,WD,Normal,181500


In [53]:
dataset.shape

(1460, 77)

In [54]:
dataset.drop('Id',inplace = True,axis=1)

In [55]:
dataset.head(2)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500


In [56]:
original_features=list(dataset.columns)
len(original_features)

76

It is zero as now everything has been filled

In [57]:
features_missing_values=list(dataset.columns[dataset.isna().any()])
len(features_missing_values)

15

In [58]:
len(dataset)

1460

# Filling Missing Features

In [59]:
bfill,mean_fill=[],[]

In [60]:
for feature in features_missing_values:
  print(feature,' ',dataset[feature].isna().sum()/len(dataset)*100,' ',dataset[feature].dtype)
  if dataset[feature].dtype=='float64':
    mean_fill.append(feature)
  else:
    bfill.append(feature)

LotFrontage   17.73972602739726   float64
MasVnrType   0.547945205479452   object
MasVnrArea   0.547945205479452   float64
BsmtQual   2.5342465753424657   object
BsmtCond   2.5342465753424657   object
BsmtExposure   2.6027397260273974   object
BsmtFinType1   2.5342465753424657   object
BsmtFinType2   2.6027397260273974   object
Electrical   0.0684931506849315   object
FireplaceQu   47.26027397260274   object
GarageType   5.5479452054794525   object
GarageYrBlt   5.5479452054794525   float64
GarageFinish   5.5479452054794525   object
GarageQual   5.5479452054794525   object
GarageCond   5.5479452054794525   object


In [61]:
mean_fill

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [62]:
for feature in mean_fill:
  dataset[feature].fillna(dataset[feature].mean(),inplace=True)

In [63]:
bfill

['MasVnrType',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond']

In [64]:
for feature in bfill:
  dataset[feature].fillna(method='bfill',inplace=True)

In [65]:
dataset.columns[dataset.isna().any()]

Index(['FireplaceQu'], dtype='object')

In [66]:
dataset['FireplaceQu'].isna().sum()

2

In [67]:
dataset['FireplaceQu'].fillna(method='ffill',inplace=True)

In [68]:
dataset.head(1)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500


# Encoding Dataset

In [69]:
categorical_features=list(dataset.select_dtypes(include=['object']).columns)
categorical_features

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [70]:
numerical_features=list(set(original_features)-set(categorical_features))
numerical_features

['OverallQual',
 'YearBuilt',
 'BsmtFinSF2',
 'SalePrice',
 'ScreenPorch',
 'MiscVal',
 'TotRmsAbvGrd',
 'LotArea',
 'TotalBsmtSF',
 'OverallCond',
 'LowQualFinSF',
 'WoodDeckSF',
 'YearRemodAdd',
 'MoSold',
 'KitchenAbvGr',
 'EnclosedPorch',
 'LotFrontage',
 'GarageCars',
 'FullBath',
 'BsmtHalfBath',
 'MSSubClass',
 '1stFlrSF',
 'YrSold',
 'BsmtUnfSF',
 'HalfBath',
 'BedroomAbvGr',
 'MasVnrArea',
 'OpenPorchSF',
 'BsmtFullBath',
 '3SsnPorch',
 'GarageYrBlt',
 'GarageArea',
 'PoolArea',
 'BsmtFinSF1',
 '2ndFlrSF',
 'GrLivArea',
 'Fireplaces']

In [71]:
nominal=['MSZoning','LandContour','Neighborhood']
ordinal=list(set(categorical_features)-set(nominal))

In [72]:
target=['SalePrice']

In [73]:
dataset_nominal=pd.get_dummies(dataset[nominal])

In [74]:
for feature in ordinal:
  dataset[feature]=dataset[feature].astype('category').cat.codes

dataset_ordinal=dataset[ordinal]

In [75]:
dataset[numerical_features]

,OverallQual,YearBuilt,BsmtFinSF2,SalePrice,ScreenPorch,MiscVal,TotRmsAbvGrd,LotArea,TotalBsmtSF,OverallCond,...,OpenPorchSF,BsmtFullBath,3SsnPorch,GarageYrBlt,GarageArea,PoolArea,BsmtFinSF1,2ndFlrSF,GrLivArea,Fireplaces
0,7,2003,0,208500,0,0,8,8450,856,5,...,61,1,0,2003.0,548,0,706,854,1710,0
1,6,1976,0,181500,0,0,6,9600,1262,8,...,0,0,0,1976.0,460,0,978,0,1262,1
2,7,2001,0,223500,0,0,6,11250,920,5,...,42,1,0,2001.0,608,0,486,866,1786,1
3,7,1915,0,140000,0,0,7,9550,756,5,...,35,1,0,1998.0,642,0,216,756,1717,1
4,8,2000,0,250000,0,0,9,14260,1145,5,...,84,1,0,2000.0,836,0,655,1053,2198,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,6,1999,0,175000,0,0,7,7917,953,5,...,40,0,0,1999.0,460,0,0,694,1647,1
1456,6,1978,163,210000,0,0,7,13175,1542,6,...,0,1,0,1978.0,500,0,790,0,2073,2
1457,7,1941,0,266500,0,2500,9,9042,1152,9,...,60,0,0,1941.0,252,0,275,1152,2340,2
1458,5,1950,1029,142125,0,0,5,9717,1078,6,...,0,1,0,1950.0,240,0,49,0,1078,0


In [76]:
new_dataset=pd.concat([dataset_nominal,dataset_ordinal,dataset[numerical_features]],axis=1)

In [77]:
print(new_dataset.shape)

(1460, 107)


# Standardization 

In [78]:
new_dataset.drop('SalePrice',axis=1,inplace=True)

In [79]:
X=new_dataset.to_numpy()

In [80]:
X[0]

array([0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 4.000e+00, 4.000e+00,
       1.000e+00, 5.000e+00, 3.000e+00, 4.000e+00, 4.000e+00, 1.000e+00,
       2.000e+00, 2.000e+00, 2.000e+00, 4.000e+00, 0.000e+00, 4.000e+00,
       2.000e+00, 2.000e+00, 0.000e+00, 1.300e+01, 0.000e+00, 1.000e+00,
       1.000e+00, 3.000e+00, 4.000e+00, 2.000e+00, 1.000e+00, 1.000e+00,
       6.000e+00, 8.000e+00, 2.000e+00, 0.000e+00, 1.000e+00, 3.000e+00,
       1.200e+01, 2.000e+00, 5.000e+00, 1.000e+00, 7.000e+00, 2.003e+03,
       0.000e+00, 0.000e+00, 0.000e+00, 8.000e+00, 8.450e+03, 8.560e+02,
       5.000e+00, 0.000e+00, 0.000e+00, 2.003e+03, 

In [81]:
scale_X = StandardScaler()
X=scale_X.fit_transform(X)

In [82]:
X[0]

array([-0.08304548, -0.21585871, -0.10526316,  0.51813339, -0.41895507,
       -0.21235968, -0.18831089, -0.15899968,  0.33712564, -0.10854037,
       -0.03703704, -0.10526316, -0.20339487, -0.1398323 ,  2.95522137,
       -0.19025216, -0.27116307, -0.23917551, -0.16124951, -0.10854037,
       -0.1863522 , -0.42683279, -0.07875671, -0.22941573, -0.16998114,
       -0.23595776, -0.28963792, -0.13199092, -0.23106504, -0.20521398,
       -0.25018188, -0.13199092, -0.16347148, -0.08712888,  0.303692  ,
        0.22403433, -0.12304604,  0.30941909,  0.75073056,  0.2085023 ,
        0.60466978, -0.12579688,  0.28974476, -0.03628912, -0.40936914,
        0.18797343, -0.22571613,  1.11993319, -0.40779461, -0.29055229,
       -0.88465767,  0.75162623, -0.02618016, -0.23012211,  0.06423821,
        0.28780036,  0.36420746,  0.83559903,  0.26381257, -0.49151573,
        0.25597761,  0.31386709, -0.03174026, -0.41169079, -1.23804229,
        0.64046232,  0.74309239, -0.77797579,  1.02668924, -0.71

In [83]:
y=dataset[target]

In [84]:
y=y.to_numpy()
y

array([[208500],
       [181500],
       [223500],
       ...,
       [266500],
       [142125],
       [147500]])

In [85]:
y.shape

(1460, 1)

In [86]:
y

array([[208500],
       [181500],
       [223500],
       ...,
       [266500],
       [142125],
       [147500]])

In [87]:
scale_y = StandardScaler()
y=scale_y.fit_transform(y)

In [88]:
y

array([[ 0.34727322],
       [ 0.00728832],
       [ 0.53615372],
       ...,
       [ 1.07761115],
       [-0.48852299],
       [-0.42084081]])

In [89]:
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.2, shuffle = False)

In [90]:
X.shape

(1460, 106)

# Feature Extraction Using PCA

In [91]:
pca=PCA(n_components=2)
pca_test=PCA(n_components=2)

In [92]:
p_components=pca.fit_transform(X_train)
p_components_test=pca_test.fit_transform(X_test)

In [93]:
p_components

array([[ 2.50219088, -1.04114291],
       [ 0.30530224,  2.79302909],
       [ 2.91452285, -0.57283085],
       ...,
       [ 3.5029916 , -1.36671181],
       [ 3.86594596, -0.59988616],
       [ 2.25711872, -0.38372006]])

In [94]:
pca.explained_variance_ratio_

array([0.10470377, 0.0448469 ])

In [95]:
pca_data = pd.DataFrame(p_components,columns=['PC1','PC2'])
target_data = pd.DataFrame(y_train,columns=['SalePrice'])

pca_test_data = pd.DataFrame(p_components_test,columns=['PC1','PC2'])
target_test_data = pd.DataFrame(y_test,columns=['SalePrice'])

final_data = pd.concat([pca_data,target_data],axis=1)
final_data

,PC1,PC2,SalePrice
0,2.502191,-1.041143,0.347273
1,0.305302,2.793029,0.007288
2,2.914523,-0.572831,0.536154
3,-1.345758,0.433312,-0.515281
4,5.169639,-0.257645,0.869843
...,...,...,...
1163,-2.326348,1.635372,-0.906150
1164,1.134931,2.663904,0.164689
1165,3.502992,-1.366712,0.657919
1166,3.865946,-0.599886,0.811290


# Installing Tensorflow

In [ ]:
pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install tf-nightly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# ANN Model

In [96]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Dense(3, input_shape=(2,), activation='tanh'),
    tf.keras.layers.Dense(4, activation = 'tanh'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',loss = 'mean_squared_error')

model.fit(pca_data, target_data, epochs=10, batch_size=2)


Epoch 1/10
584/584 [==============================] - 1s 1ms/step - loss: 0.5321
Epoch 2/10
584/584 [==============================] - 1s 1ms/step - loss: 0.4319
Epoch 3/10
584/584 [==============================] - 1s 1ms/step - loss: 0.3607
Epoch 4/10
584/584 [==============================] - 1s 1ms/step - loss: 0.3055
Epoch 5/10
584/584 [==============================] - 1s 1ms/step - loss: 0.2679
Epoch 6/10
584/584 [==============================] - 1s 1ms/step - loss: 0.2427
Epoch 7/10
584/584 [==============================] - 1s 1ms/step - loss: 0.2245
Epoch 8/10
584/584 [==============================] - 1s 1ms/step - loss: 0.2150
Epoch 9/10
584/584 [==============================] - 1s 1ms/step - loss: 0.2070
Epoch 10/10
584/584 [==============================] - 1s 1ms/step - loss: 0.1998


In [97]:
loss = model.evaluate(pca_test_data, target_test_data,batch_size = 2)
print(loss)

146/146 [==============================] - 0s 1ms/step - loss: 0.2694
0.269413560628891


In [98]:
predictions =model.predict(pca_test_data)
predictions = scale_y.inverse_transform(predictions)
features = pca_test_data
output_y = pd.DataFrame(predictions,columns=['Predictions'])
output = pd.concat([features,output_y],axis=1)

output



,PC1,PC2,Predictions
0,0.210558,2.372315,169461.875000
1,9.411013,3.479734,397635.687500
2,1.568859,4.877584,215390.343750
3,-0.942123,0.724642,142540.312500
4,0.569310,-4.985899,156208.437500
...,...,...,...
287,0.814534,-1.770234,165617.218750
288,1.357379,1.765891,188661.859375
289,1.185217,1.748692,184677.000000
290,-2.762836,-0.017426,119280.179688
